<a href="https://colab.research.google.com/github/gskumlehn/french-teacher/blob/main/wikipedia_search_audio_fr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar dependências no Colab
!apt-get -y update && apt-get -y install ffmpeg portaudio19-dev
!pip install -q gTTS wikipedia googletrans==4.0.0-rc1 SpeechRecognition pydub

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [2]:
import os
import wikipedia
from gtts import gTTS
from googletrans import Translator
import speech_recognition as sr
from pydub import AudioSegment
from IPython.display import Audio, display

In [3]:
def speak_fr(text, filename="response_fr.mp3"):
    if os.path.exists(filename):
        os.remove(filename)
    tts = gTTS(text=text, lang='fr')
    tts.save(filename)
    return filename

def transcribe_en(audio_path, language='en-US'):
    r = sr.Recognizer()
    ext = os.path.splitext(audio_path)[1].lower()
    wav_path = audio_path
    if ext not in ['.wav', '.aiff', '.aif']:
        wav_path = os.path.splitext(audio_path)[0] + '.wav'
        AudioSegment.from_file(audio_path).export(wav_path, format='wav')
    with sr.AudioFile(wav_path) as source:
        audio = r.record(source)
    return r.recognize_google(audio, language=language)

In [4]:
wikipedia.set_lang("en")
translator = Translator()

audio_path = '/content/strawberry.wav' #record_audio(duration=5)

try:
    query_text = transcribe_en(audio_path, language='en-US')
    print(f"You said: {query_text}")
except Exception as e:
    print(f"Error: {e}")
    query_text = ""

if query_text:
    keyword = query_text.strip().split()[0]
    try:
        summary_en = wikipedia.summary(keyword, sentences=3, auto_suggest=True, redirect=True)
        summary_fr = translator.translate(summary_en, src='en', dest='fr').text
        print("French summary:")
        print(summary_fr)
        mp3_path = speak_fr(summary_fr)
        display(Audio(mp3_path))
    except Exception as e:
        print(f"Error: {e}")

You said: strawberry
French summary:
La fraise du jardin (ou simplement la fraise; fragaria × ananassa) est une plante hybride largement cultivée cultivée dans le monde entier pour ses fruits.Le genre Fragaria, les fraises, se trouve dans la famille Rose, Rosaceae.Le fruit est apprécié pour son arôme, sa couleur rouge vif, sa texture juteuse et sa douceur.
